In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import json
import pandas as pd
import numpy as np
!pip install transformers
import torch
import torch.nn.functional as F
device=torch.device('cuda')
from transformers import BertTokenizer,BertForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
from lxml import etree as ET
!pip3 install scispacy
!pip3 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
import spacy
import scispacy
import en_core_sci_lg
nlp = en_core_sci_lg.load()
from bs4 import BeautifulSoup

"""
#Create input.csv file from training8b.json file
jsonFile = open('/content/gdrive/My Drive/Colab Notebooks/BioASQ/training8b.json', 'r')
values = json.load(jsonFile)
jsonFile.close()

id_list = []
question_list = []
for i in range(len(values['questions'])):
    id_list.append(values['questions'][i]['id'])
    question_list.append(values['questions'][i]['body'])

input_df =  pd.DataFrame(list(zip(id_list,question_list)),columns=['ID','Question'])
input_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/BioASQ/input.csv',index=False)  
"""


     |████████████████████████████████| 778kB 6.4MB/s 
     |████████████████████████████████| 3.0MB 18.1MB/s 
     |████████████████████████████████| 1.1MB 41.3MB/s 
     |████████████████████████████████| 890kB 36.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4292aea4a62942486e3059d31c64ab18545b1f89e682184c7bf3b3d6c565a5ca
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 10.0MB 6.5MB/s 
     |████████████████████████████████| 13.0MB 239kB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 2.1MB 37.5MB/s 
     |████████████████████████████████| 296kB 36.4MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Succes

/usr/local/lib/python3.6/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_sci_lg' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


"\njsonFile = open('/content/gdrive/My Drive/Colab Notebooks/BioASQ/training8b.json', 'r')\nvalues = json.load(jsonFile)\njsonFile.close()\n\nid_list = []\nquestion_list = []\nfor i in range(len(values['questions'])):\n    id_list.append(values['questions'][i]['id'])\n    question_list.append(values['questions'][i]['body'])\n\ninput_df =  pd.DataFrame(list(zip(id_list,question_list)),columns=['ID','Question'])\ninput_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/BioASQ/input.csv',index=False)  \n"

Read input file (.csv) and predict type for each question

In [3]:
def preprocess(df):
  df.encoded_tokens = [tokenizer.encode_plus(text,add_special_tokens=True)['input_ids'] for text in df['Question']] #encoded tokens for each tweet
  df.attention_mask = [tokenizer.encode_plus(text,add_special_tokens=True)['attention_mask'] for text in df['Question']]
  encoded_tokens = list(df.encoded_tokens)
  attention_mask = list(df.attention_mask)
  return encoded_tokens,attention_mask

# Convert indices to Torch tensor and dump into cuda
def feed_generator(encoded_tokens,attention_mask):

    batch_size = 16
    batch_seq = [x for x in range(int(len(encoded_tokens)/batch_size))]


    shuffled_encoded_tokens,shuffled_attention_mask = encoded_tokens,attention_mask

    res = len(encoded_tokens)%batch_size
    if res != 0:
        batch_seq = [x for x in range(int(len(encoded_tokens)/batch_size)+1)]
    shuffled_encoded_tokens = shuffled_encoded_tokens+shuffled_encoded_tokens[:res]
    shuffled_attention_mask = shuffled_attention_mask+shuffled_attention_mask[:res]

    for batch in batch_seq:
        maxlen_sent = max([len(i) for i in shuffled_encoded_tokens[batch*batch_size:(batch+1)*batch_size]])
        token_tensor = torch.tensor([tokens+[0]*(maxlen_sent-len(tokens)) for tokens in shuffled_encoded_tokens[batch*batch_size:(batch+1)*batch_size]])
        attention_mask = torch.tensor([tokens+[0]*(maxlen_sent-len(tokens)) for tokens in shuffled_attention_mask[batch*batch_size:(batch+1)*batch_size]]) 

        token_tensor = token_tensor.to('cuda')
        attention_mask = attention_mask.to('cuda')

        yield token_tensor,attention_mask

def predict(model,data):
    model.eval()
    model.cuda()
    preds = []
    batch_count = 0
    for token_tensor, attention_mask in data:
        with torch.no_grad():
            logits = model(token_tensor,token_type_ids=None,attention_mask=attention_mask)[0]
        tmp_preds = torch.argmax(logits,-1).detach().cpu().numpy().tolist()
        preds += tmp_preds             
    return preds

test_data_path = '/content/gdrive/My Drive/Colab Notebooks/BioASQ/input.csv'
testing_df = pd.read_csv(test_data_path,sep=',',header=0)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
checkpoint_dir = "gdrive/My Drive/Colab Notebooks/bert-large-v3/"


model = BertForSequenceClassification.from_pretrained('/content/gdrive/My Drive/Colab Notebooks/BioASQ/Model/model 1/', cache_dir=None)

encoded_tokens_Test,attention_mask_Test = preprocess(testing_df)
data_test = feed_generator(encoded_tokens_Test, attention_mask_Test)
preds_test = predict(model,data_test)


indices_to_label = {0: 'factoid', 1: 'list', 2: 'summary', 3: 'yesno'}

predict_label = []
for i in preds_test[0:len(testing_df['Question'])]:
  for j in indices_to_label:
    if i == j:
      predict_label.append(indices_to_label[j])

testing_df['type'] = predict_label


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


Create output file (XML) providing question type to Answer Processing system and query for Information Retrieval system

In [4]:
def xml_tree(df):
    root = ET.Element("Input")
    for ind in df.index:
      id = df['ID'][ind]
      question = df['Question'][ind]
      qtype = df['type'][ind]
      q = ET.SubElement(root,"Q")
      q.set('id',str(id))
      q.text = question
      qp = ET.SubElement(q,"QP")
      qp_type = ET.SubElement(qp,'Type')
      qp_type.text = qtype
      doc = nlp(question)
      ent_list = []
      for ent in doc.ents:
        ent_list.append(str(ent))
        qp_en = ET.SubElement(qp,'Entities') 
        qp_en.text = str(ent)
      qp_query = ET.SubElement(qp,'Query')
      qp_query.text = str(' '.join(ent_list))
      
    tree = ET.ElementTree(root)
    tree.write('gdrive/My Drive/Colab Notebooks/BioASQ/qp_demo.xml', pretty_print=True)
    

xml_tree(testing_df)    
    